In [41]:
import sys
import os

# Get the path of the current notebook's directory and append the parent directory to the path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import importlib

import pdc_functions.assumption_utils as assumptions
import pdc_functions.data_utils as data
import pdc_functions.explore_utils as explore
import pdc_functions.pdc_1and2_utils as pdc
import pdc_functions.stats_utils as stats
from dotenv import load_dotenv
import pdc_functions.class_utils as class_utils
import pandas as pd

importlib.reload(class_utils)

from phmlondon.snow_utils import SnowflakeConnection


In [6]:
load_dotenv()

snowsesh = SnowflakeConnection()
snowsesh.use_database("INTELLIGENCE_DEV")
snowsesh.use_schema("AI_CENTRE_FEATURE_STORE")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/8076439c-5b1f-4e15-91ea-a0bff0b3bf16/saml2?SAMLRequest=nZJfb9owFMW%2FSuQ9J7HDnxILqCgBDQk6BrTd9mYSBywcO%2FN1GrpPPyeA1D20D5PyENnn3t%2FxPXd4fy6k98oNCK1GiAQYeVylOhPqMEJPu7k%2FQB5YpjImteIj9MYB3Y%2BHwApZ0kllj2rDf1ccrOcaKaDNxQhVRlHNQABVrOBAbUq3k9WSRgGmDIAb63DoWpKBcKyjtSUNw7qug7oTaHMII4xxiOPQqRrJF%2FQOUX7OKI22OtXyVnJ2b%2FoAQULcbRBO4Qjra%2BGDUJcRfEbZX0RAv%2B52a3%2F9bbtD3uT2uqlWUBXcbLl5FSl%2F2iwvBsA52M56fTLoB9XJB%2B1mGLA%2FleEBKF3nkp14qouysq514P7CnGeh1AfhBrZIRqg8iUwS%2BV3O8Y8HmEqVH80zO09%2FJi%2FnlXlJipWZzOTmLlkvT6fNLEXe8y3eqIl3AVDxhWpCte4IRz0fuy%2Fe4Zj2YtrtBBEZ%2FEJe4kIVitm28ua89REUIjUadG61kkLx1uUA3%2FW7nTj1e3uS%2B11Oen5MOPMZ3uc53nf2OemHTXQRuqwPbY2Y8X8NZRi%2Bb3Fdx0eX0CJZaynSN2%2BuTcHsxwGSgLQnIvPzVkp5wYScZJnhAC5IKXU9NZ

In [21]:
ldl = data.get_data_by_cohort(snowsesh, "intelligence_dev.ai_centre_dev.ldl_all")


Retrieved 8775928 rows from cohort table: intelligence_dev.ai_centre_dev.ldl_all


In [38]:
statin = data.get_data_by_cohort(snowsesh, "intelligence_dev.ai_centre_dev.statin_all")

Retrieved 13686879 rows from cohort table: intelligence_dev.ai_centre_dev.statin_all


In [42]:
drug_table = class_utils.MedicationTable(statin)


In [43]:
drug_table.validate_columns()  # Validate columns
drug_table.normalise_missing_values()  # Standardise representation of missing data
drug_table.convert_dates()  # Convert date columns
drug_table.validate_data_types()  # Validate data types
drug_table.clean_dose()  # Clean and interpret the 'dose' column
drug_table.calculate_covered_days()

print("All validations passed successfully!")

Invalid rows (dose or tablets_per_day issue):
Empty DataFrame
Columns: [dose, tablets_per_day]
Index: []
All validations passed successfully!


In [ ]:
person_drug_pdc = pdc.compute_pdc_overall(drug_table.df)